In [1]:
###### phase 7: re-check gene annotations, comparing it to the GBK, filter out ones without any new information added ######
#### it also populate organism and taxon properties from mibig finalgbk ####

In [2]:
from os import path, makedirs
import glob
import json
from jsonschema import validate, Draft7Validator
from tempfile import TemporaryDirectory
from Bio import SeqIO, Entrez
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Alphabet import generic_protein
from Bio.SeqRecord import SeqRecord
import sys
import datetime
import time
import re
import urllib3
from urllib.parse import quote
import certifi
from xml.dom import minidom
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
from jsonschema.validators import Draft7Validator

In [3]:
def get_ncbi_tax_ids(organism_name, email):
    tax_ids = []
    
    Entrez.email = email
    num_try = 1
    while num_try < 6:
        try:
            organism_name_escaped = "\"{}\"".format(organism_name)
            dom = minidom.parse(Entrez.esearch(db="taxonomy", term=organism_name_escaped))
            ids = dom.getElementsByTagName('Id')
            for tax_id in ids:
                tax_ids.append(tax_id.firstChild.nodeValue)
            break
        except:
            pass
        num_try += 1
        time.sleep(5)
    
    return tax_ids

def fetch_gbk(nucl_acc, email, clean_cache = False):
    cache_folder = "../../preprocessed/cache/cached_gbks/"
    cache_path = path.join(cache_folder, "{}.gbk".format(nucl_acc.replace(":", ".")))

    if not path.exists(cache_folder):
        makedirs(cache_folder)
    
    if clean_cache or not path.exists(cache_path) or path.getsize(cache_path) < 100:
        if re.match("^MIBIG:BGC\d{7}\.\d+$", nucl_acc):
            bgc_id = nucl_acc.split(":")[1].split(".")[0]
            cl_id = nucl_acc.split(":")[1].split(".")[1]
            mibig_url = "https://mibig.secondarymetabolites.org/repository/{}/{}.1.cluster{:03d}.gbk".format(bgc_id, bgc_id, int(cl_id))
            resp = http.request('GET', mibig_url)
            with open(cache_path, "w") as gbk_file:
                gbk_file.write(resp.data.decode('utf-8', 'ignore'))
        else: # (re)download from ncbi
            Entrez.email = email
            handle = Entrez.efetch(db="nucleotide", id=nucl_acc, rettype="gbwithparts", retmode="text")
            if not path.exists(cache_folder):
                makedirs(cache_folder)
            with open(cache_path, "w") as gbk_file:
                gbk_file.write(handle.read())
            
    return open(cache_path, "r")

# from antismash
def get_aa_translation(seq_record, feature):
    """Obtain content for translation qualifier for specific CDS feature in sequence record"""
    extracted = feature.extract(seq_record.seq).ungap('-')

    # ensure the extracted section is a multiple of three by trimming any excess
    if len(extracted) % 3 != 0:
        extracted = extracted[:-(len(extracted) % 3)]

    fasta_seq = extracted.translate(to_stop=True)
    if len(fasta_seq) == 0:
        print("Retranslating {} with stop codons".format(feature.id))
        fasta_seq = extracted.translate()

    # replace ambiguous aminos with an explicit unknown
    string_version = str(fasta_seq)
    for bad in "*BJOUZ":
        string_version = string_version.replace(bad, "X")

    # and remove any gaps
    string_version = string_version.replace("-", "")
    fasta_seq = Seq(string_version, generic_protein)

    return fasta_seq

def get_aa_sequence(feature, to_stop=False):
    """Extract sequence from specific CDS feature in sequence record"""
    fasta_seq = feature.qualifiers['translation'][0]
    if "*" in fasta_seq:
        if to_stop:
            fasta_seq = fasta_seq.split('*')[0]
        else:
            fasta_seq = fasta_seq.replace("*","X")
    if "-" in fasta_seq:
        fasta_seq = fasta_seq.replace("-","")
    return fasta_seq


def count_props(input_dict, cur_path, result):
    """given a (mibig?) json, construct a list of property paths
    along with its presence count in the json object"""
    key_path = cur_path
    
    if isinstance(input_dict, dict):
        for key in input_dict.keys():
            result = count_props(input_dict[key], "{}/{}".format(key_path, key), result)
    elif isinstance(input_dict, list):
        key_path = "{}[]".format(key_path)
        for node in input_dict:
            result = count_props(node, "{}".format(key_path), result)

    if not isinstance(input_dict, dict):
        if key_path not in result:
            result[key_path] = 0
        result[key_path] += 1
    
    return result


def fetch_props_new_schema(input_dict, cur_path, result):
    """given a (mibig?) json draft7 schema, construct a list of property paths
    along with either required == True for each properties"""
    key_path = cur_path
    if ("type" not in input_dict) or (input_dict["type"] not in ["object", "array"]):
        key_path = "{}".format(cur_path) # string / etc.
    elif input_dict["type"] == "object":
        for key in input_dict["properties"]:
            result = fetch_props_new_schema(input_dict["properties"][key], "{}/{}".format(key_path, key), result)
    elif input_dict["type"] == "array":
        key_path = "{}[]".format(cur_path)
        result = fetch_props_new_schema(input_dict["items"], "{}".format(key_path), result)
    
    if key_path not in result and "properties" not in input_dict:
        result[key_path] = False # can't really use this
    return result


In [4]:
def fetch_mibig_final_gbk(bgc_id, clean_cache = False):
    cache_folder = "../../preprocessed/cache/cached_mibig_finalgbks/"
    cache_path = path.join(cache_folder, "{}.1.final.gbk".format(bgc_id))
    
    if clean_cache or not path.exists(cache_path) or path.getsize(cache_path) < 100:
        mibig_url = "https://mibig.secondarymetabolites.org/repository/{}/{}.1.final.gbk".format(bgc_id, bgc_id)
        resp = http.request('GET', mibig_url)
        with open(cache_path, "w") as cf:
            cf.write(resp.data.decode('utf-8', 'ignore'))
        
    return open(cache_path, "r")

In [5]:
def _fetch_val_to_ref(input_dict, parent_dict, key, all_references):
    if isinstance(input_dict, dict):
        for key in input_dict.keys():
            _fetch_val_to_ref(input_dict[key], input_dict, key, all_references)
    elif isinstance(input_dict, list):
        for i, node in enumerate(input_dict):
            _fetch_val_to_ref(node, input_dict, i, all_references)
    elif isinstance(input_dict, str):
        if parent_dict != None:
            valu = input_dict.upper()
            if valu not in all_references:
                all_references[valu] = []
            all_references[valu].append((parent_dict, key))
    return all_references

with open("../../preprocessed/reports/p7-updated_cases.tsv", "w") as uc:
    uc.write("")
    
def _make_gene_id_match_reference(input_dict, reference_ids, bgc_id):
    all_references = _fetch_val_to_ref(input_dict, None, None, {})
    all_rids = {}
    for rid in reference_ids:
        if len(rid) > 0:
            all_rids[rid.upper()] = rid
    matches = set(all_references.keys()).intersection(set(all_rids.keys()))    
    for key in matches:
        for ar in all_references[key]:
            if ar[0][ar[1]] != all_rids[key]:
                with open("../../preprocessed/reports/p7-updated_cases.tsv", "a") as uc:
                    uc.write("{}\t{}\t{}\n".format(bgc_id, ar[0][ar[1]], all_rids[key]))
                ar[0][ar[1]] = all_rids[key]

def make_gene_id_match_reference(data, reference_ids):
    bgc_id = data["cluster"]["mibig_accession"]
    _make_gene_id_match_reference(data["cluster"]["compounds"], reference_ids, bgc_id)
    _make_gene_id_match_reference(data["cluster"].get("genes", {}), reference_ids, bgc_id)
    _make_gene_id_match_reference(data["cluster"].get("polyketide", {}), reference_ids, bgc_id)
    _make_gene_id_match_reference(data["cluster"].get("nrp", {}), reference_ids, bgc_id)
    _make_gene_id_match_reference(data["cluster"].get("ripp", {}), reference_ids, bgc_id)
    _make_gene_id_match_reference(data["cluster"].get("terpene", {}), reference_ids, bgc_id)
    _make_gene_id_match_reference(data["cluster"].get("saccharide", {}), reference_ids, bgc_id)
    
    return data

In [6]:
removed_extra_genes = {}
removed_annotations = {}
removed_operons = {}
no_taxid = {}
added_mibig_genes = {}
no_gbk = {}
fixed = {}
ripps_to_fix = {}

def check_gene_annotations(data):
    loci = data["cluster"]["loci"]
    annots = data["cluster"].get("genes", {})
    gbk_acc = loci["accession"].upper()
    bgc_id = data["cluster"]["mibig_accession"]
    email = "mibig@secondarymetabolites.org"
    gbk_record = None
    if True:
        num_try = 1
        clean_cache = False
        while num_try < 6:
            try:
                with fetch_gbk(gbk_acc, email, clean_cache) as gbk_handle:
                    seq_record = SeqIO.read(gbk_handle, "genbank") # the gbk should contains only 1 file
                    if len(seq_record.seq) < 1:
                        raise Exception("Empty sequence record {}".format(gbk_acc))
                    gbk_record = seq_record
                    break
            except:
                print("Error...")
                clean_cache = True
            num_try += 1
            time.sleep(5)
        if not isinstance(gbk_record, SeqRecord): # failed to download NCBI data
            print("{} Failed to download: {}".format(data["cluster"]["mibig_accession"], gbk_acc))
            no_gbk[bgc_id] = gbk_acc
            return {}

    # fill up taxid
    tax_ids = get_ncbi_tax_ids(gbk_record.annotations["organism"], email)
    if len(tax_ids) != 1:
        no_taxid[bgc_id] = gbk_record.annotations["organism"]
        print(tax_ids)
    else:
        data["cluster"]["ncbi_tax_id"] = tax_ids[0]
        data["cluster"]["organism_name"] = gbk_record.annotations["organism"]
         
    # fetch all CDS inside the cluster
    cluster_cds = {}
    cluster_cds_ids = set(["", "No protein ID"]) # quick lookup for existing IDs
    for feature in seq_record.features:
        if feature.type == "CDS":
            if ("start_coord" not in loci) or (feature.location.start >= loci["start_coord"]-1 and feature.location.end <= loci["end_coord"]):
                if "gene" in feature.qualifiers:
                    cluster_cds_ids.add(feature.qualifiers["gene"][0])
                    cluster_cds[feature.qualifiers["gene"][0]] = feature
                if "protein_id" in feature.qualifiers:
                    cluster_cds_ids.add(feature.qualifiers["protein_id"][0])
                    cluster_cds[feature.qualifiers["protein_id"][0]] = feature
                if "locus_tag" in feature.qualifiers:
                    cluster_cds_ids.add(feature.qualifiers["locus_tag"][0])
                    cluster_cds[feature.qualifiers["locus_tag"][0]] = feature
                
    # fetch all CDS in MIBiG finalgbk
    mibig_extra_genes = []
    with fetch_mibig_final_gbk(bgc_id) as mibig_gbk_handle:
        mibig_seq_records = SeqIO.parse(mibig_gbk_handle, "genbank")
        for mibig_seq_record in mibig_seq_records:
            # look for gene information
            match = re.search("(between(?P<start>\d+)-(?P<end>\d+)ntfrom){0,1}GenBankID(?P<acc>[A-Z0-9_\.]+)\.", mibig_seq_record.annotations['comment'].replace(" ", "").replace("\n", ""))
            if match:
                gbk_acc_mibig = match.group("acc").upper()
                if gbk_acc_mibig == gbk_acc:
                    offset = 0
                    if match.group("start"):
                        offset = int(match.group("start")) - 1
                    for feature in mibig_seq_record.features:
                        if feature.type == "CDS":
                            ids = []
                            if "gene" in feature.qualifiers:
                                if feature.qualifiers["gene"][0] in cluster_cds_ids:
                                    continue
                                ids.append(feature.qualifiers["gene"][0])
                                cluster_cds[feature.qualifiers["gene"][0]] = feature
                            if "protein_id" in feature.qualifiers:
                                if feature.qualifiers["protein_id"][0] in cluster_cds_ids:
                                    continue
                                ids.append(feature.qualifiers["protein_id"][0])
                                cluster_cds[feature.qualifiers["protein_id"][0]] = feature
                            if "locus_tag" in feature.qualifiers:
                                if feature.qualifiers["locus_tag"][0] in cluster_cds_ids:
                                    continue
                                ids.append(feature.qualifiers["locus_tag"][0])
                                cluster_cds[feature.qualifiers["locus_tag"][0]] = feature
                            cluster_cds_ids.update(set(ids))
                            mibig_extra_genes.append(feature._shift(offset)) 
                            if bgc_id not in added_mibig_genes:
                                added_mibig_genes[bgc_id] = []
                            added_mibig_genes[bgc_id].append("/".join([gid for gid in ids]))
                            
    # add mibig extra genes
    for feature in mibig_extra_genes:
        if "extra_genes" not in annots:
            annots["extra_genes"] = []
        if "annotations" not in annots:
            annots["annotations"] = []
        gene_id = feature.qualifiers.get("locus_tag", feature.qualifiers.get("protein_id", feature.qualifiers.get("gene", [None])))[0]
        if gene_id != None:
            extra_gene = {
                "id": gene_id,
                "location": {
                    "exons": [{"start": location.start + 1, "end": location.end} for location in feature.location.parts],
                    "strand": feature.location.strand
                }
            }
            if "translation" in feature.qualifiers:
                extra_gene["translation"] = feature.qualifiers["translation"][0]
            annots["extra_genes"].append(extra_gene)
            annot = {
                "id": gene_id
            }
            if "product" in feature.qualifiers:
                annot["product"] = feature.qualifiers["product"][0]
            if "note" in feature.qualifiers:
                annot["comments"] = feature.qualifiers["note"][0]
            if len(annot.keys()) > 1:
                annots["annotations"].append(annot)
                
    # check extra genes annotation
    extra_genes = []
    for i, extra_gene in enumerate(annots.get("extra_genes", [])):
        gene_id = extra_gene["id"]
        if gene_id.upper() not in [gid.upper() for gid in cluster_cds_ids] or gene_id in [gid.upper() for gid in "/".join(added_mibig_genes.get(bgc_id, "")).split("/")]:
            extra_genes.append(extra_gene)
            cluster_cds_ids.add(gene_id)
        else:
            if bgc_id not in removed_extra_genes:
                removed_extra_genes[bgc_id] = []
            removed_extra_genes[bgc_id].append(str(i))
    if len(extra_genes) > 0:
        annots["extra_genes"] = extra_genes
    else:
        annots.pop("extra_genes", None)
            
    # remove junk ids
    cluster_cds_ids.discard("")
    cluster_cds_ids.discard(None)
    
    # check annotation, remove if no added info
    gene_annots = []
    for i, annot in enumerate(annots.get("annotations", [])):
        approve = False
        found_id = None
        for gid in cluster_cds_ids:
            if annot.get("id", "").upper() == gid.upper():
                found_id = gid
                annot["id"] = gid
                break
            elif annot.get("name", "").upper() == gid.upper():
                found_id = gid
                annot["name"] = gid
                break
        ids = "id='{}',name='{}'".format(annot.get("id", ""), annot.get("name", ""))
        if True:
            if len(annot.get("name", "")) < 1:
                annot.pop("name", None)
            if len(annot.get("product", "")) < 1:
                annot.pop("product", None)
            if len(annot.get("mut_pheno", "")) < 1:
                annot.pop("mut_pheno", None)
            if len(annot.get("comments", "")) < 1:
                annot.pop("comments", None)
            g_functions = []
            for g_function in annot.get("functions", []):
                if len(g_function.get("category", "")) < 1:
                    continue
                elif len(g_function.get("evidence", [])) < 1:
                    continue
                elif len(set(g_function["evidence"]) - set(["Sequence-based prediction", "Other in vivo study", "Heterologous expression", "Knock-out", "Activity assay"])) > 0:
                    continue
                else:
                    g_functions.append(g_function)
            if len(g_functions) > 0:
                annot["functions"] = g_functions
            else:
                annot.pop("functions", None)
            if len(annot.get("tailoring", [])) < 1:
                annot.pop("tailoring", None)
            if len(annot.get("publications", [])) < 1:
                annot.pop("publications", None)
                
            if found_id == None:
                annot = {} # uncomment to add to extra genes when not found                
                #if len(annot.get("id", "")) < 1:
                #    if len(annot.get("name", "")) < 1:
                #        annot = {}
                #    else:
                #        annot["id"] = annot["name"]
                #if len(annot.get("id", "")) > 0:
                #    if "extra_genes" not in annots:
                #        annots["extra_genes"] = []
                #    annots["extra_genes"].append({"id": annot["id"]})
                #    cluster_cds_ids.add(annot["id"])
                    
            if len(annot.keys()) > 1: # 1 is the id
                approve = True
        if approve:
            gene_annots.append(annot)
        else:
            if bgc_id not in removed_annotations:
                removed_annotations[bgc_id] = []
            removed_annotations[bgc_id].append(ids)
    if len(gene_annots) > 0:
        annots["annotations"] = gene_annots
    else:
        annots.pop("annotations", None)
                
    # check operons
    operons = []
    for i, operon in enumerate(annots.get("operons", [])):
        o_genes = set([gid.upper() for gid in operon.get("genes", [])])
        o_evidence = set(operon.get("evidence", [])).intersection(set(["Sequence-based prediction", "RACE", "ChIPseq", "RNAseq"]))
        if len(o_genes) > 0 and len(o_genes - cluster_cds_ids) < 1:
            if len(o_evidence) > 0:
                operons.append({
                    "genes": list(o_genes),
                    "evidence": list(o_evidence)
                })
                continue
        if bgc_id not in removed_operons:
            removed_operons[bgc_id] = []
        removed_operons[bgc_id].append(str(i))
        
    if len(operons) > 0:
        annots["operons"] = operons
    else:
        annots.pop("operons", None)

    # try to fix ripp leader/follower sequences
    if "ripp" in data["cluster"]:
        fixed_precursors = []
        unfixable_precursors = []
        for i, precursor in enumerate(data["cluster"]["ripp"].get("precursor_genes", [])):
            if "gene_id" in precursor:
                cds_feature = None
                for gid in cluster_cds_ids:
                    if precursor["gene_id"].upper() == gid.upper():
                        cds_feature = cluster_cds[gid]
                        break
                if cds_feature != None:
                    if "translation" in cds_feature.qualifiers:
                        aa_sequence = str(cds_feature.qualifiers["translation"][0])
                        leader_length = precursor.get("leader_sequence", 0)
                        follower_length = precursor.get("follower_sequence", 0)
                        if leader_length < 1 or leader_length > len(aa_sequence) or follower_length > len(aa_sequence):
                            print("[ripp] Wrong leader/follower length information {}".format(precursor["gene_id"]))
                            unfixable_precursors.append(i)
                        elif len(precursor.get("core_sequence", [])) < 1:
                            print("[ripp] No core sequence {}".format(precursor["gene_id"]))
                            unfixable_precursors.append(i)
                        else:
                            cores_checked = True
                            for core in precursor["core_sequence"]:
                                if len(core) < 1:
                                    cores_checked = False
                                    break
                                core_start = aa_sequence.find(core)
                                if core_start < 0:
                                    # can't find core
                                    cores_checked = False
                                    break
                                elif core_start < leader_length or core_start + len(core) >= len(aa_sequence) - follower_length:
                                    # core overlaps with leader/follower
                                    cores_checked = False
                                    break
                            if cores_checked:
                                precursor["leader_sequence"] = aa_sequence[0:leader_length]
                                if follower_length > 0:
                                    precursor["follower_sequence"] = aa_sequence[follower_length - 1:]
                                else:
                                    precursor.pop("follower_sequence", None)
                                fixed_precursors.append(i)
                            else:
                                print("[ripp] Wrong sequence information for {}".format(precursor["gene_id"]))
                                unfixable_precursors.append(i)
                    else:
                        print("[ripp] Can't find AA sequence for {}".format(precursor["gene_id"]))
                        unfixable_precursors.append(i)
                else:
                    print("[ripp] Can't find CDS feature for {}".format(precursor["gene_id"]))
                    unfixable_precursors.append(i)
                    
        if len(fixed_precursors) > 0 and len(fixed_precursors) == len(data["cluster"]["ripp"]["precursor_genes"]):
            # remove the retired flag from ripp.leader/follower_sequence
            print("[ripp] solved precursors {}".format(bgc_id))
            if bgc_id not in fixed:
                fixed[bgc_id] = []
            fixed[bgc_id].append("ripp.leader/follower_sequence")
        elif len(unfixable_precursors) > 0:
            ripps_to_fix[bgc_id] = unfixable_precursors
            # temporarily remove the precursors, and remove the retired flag from ripp.leader/follower_sequence
            precursors = []
            for i, precursor in enumerate(data["cluster"]["ripp"]["precursor_genes"]):
                if i not in unfixable_precursors:
                    precursors.append(precursor)
            if len(precursors) > 0:
                data["cluster"]["ripp"]["precursor_genes"] = precursors
            else:
                data["cluster"]["ripp"].pop("precursor_genes", None)
            if bgc_id not in fixed:
                fixed[bgc_id] = []
            fixed[bgc_id].append("ripp.leader/follower_sequence")            
            
        
    # update gene ids in specific_infos
    data = make_gene_id_match_reference(data, cluster_cds_ids)
        
    if len(annots.keys()) > 0:
        data["cluster"]["genes"] = annots

In [7]:
def validate_data(data, validator, schema_props):
    for error in sorted(validator.iter_errors(data), key=str):
        if error.path[-2] == "evidence" or error.path[-1] == "evidence":
            if error.path[-3] == "loci" or error.path[-2] == "loci":
                continue
        elif error.path[-1] == "nr_iterations":
            continue
        elif error.path[-1] == "module_number":
            continue
        elif error.path[-2] == "proteinogenic":
            continue
        elif error.path[-1] in ["leader_sequence", "follower_sequence"]:
            continue
        elif error.path[-1] == "organism":
            if data["cluster"]["mibig_accession"] in no_gbk:
                continue
        print(error.message)
        #sys.exit(0)
    this_file_props = count_props(data, "", {})
    for prop in this_file_props:
        if prop not in schema_props.keys():
            print(prop)
            sys.exit(0)

In [8]:
input_path = "../../preprocessed/p6-json/"
output_folder = "../../preprocessed/p7-json/"

if not path.exists(output_folder):
    makedirs(output_folder)

validator = None
schema_props = {}
with open("../../inputs/mibig_schema_phase_6.json") as json_file:
    schema_obj = json.load(json_file)
    validator = Draft7Validator(schema_obj)
    schema_props = fetch_props_new_schema(schema_obj, "", {})


for json_path in sorted(glob.glob(path.join(input_path, "BGC*.json"))):
    with open(json_path, "r") as json_file:
        bgc_id = path.basename(json_path).split(".")[0]
        if int(bgc_id[3:]) < 468:
            continue
        data = json.load(json_file)
        print("Scanning {}".format(bgc_id))
        check_gene_annotations(data)
        validate_data(data, validator, schema_props)
        with open(path.join(output_folder, "{}.json".format(bgc_id)), "w") as o:
            o.write(json.dumps(data, indent=4, separators=(',', ': '), sort_keys=True))
            
print("All data fetched!")

Scanning BGC0000468
[ripp] Can't find CDS feature for bmbc,ccm09443.1
Scanning BGC0000469
Scanning BGC0000470
Scanning BGC0000471
Scanning BGC0000472
[ripp] Can't find CDS feature for ada00388
Scanning BGC0000473
[ripp] Can't find CDS feature for cao82085
Scanning BGC0000474
[ripp] Can't find CDS feature for cap64339
Scanning BGC0000475
Scanning BGC0000476
[ripp] Wrong sequence information for abi99461.1
Scanning BGC0000477
[ripp] solved precursors BGC0000477
Scanning BGC0000478
[ripp] Can't find CDS feature for eu290743
Scanning BGC0000479
[ripp] solved precursors BGC0000479
Scanning BGC0000480
[ripp] solved precursors BGC0000480
Scanning BGC0000481
Scanning BGC0000482
[ripp] Can't find CDS feature for eu290744
Scanning BGC0000483
Scanning BGC0000484
Scanning BGC0000485
Scanning BGC0000486
Scanning BGC0000487
Scanning BGC0000488
[ripp] Wrong sequence information for CAD97580.1
Scanning BGC0000489
Scanning BGC0000490
Scanning BGC0000491
Scanning BGC0000492
Scanning BGC0000493
Scanning 

Scanning BGC0000768
Scanning BGC0000769
Scanning BGC0000770
Scanning BGC0000771
Scanning BGC0000772
Scanning BGC0000773
Scanning BGC0000774
Scanning BGC0000775
Scanning BGC0000776
Scanning BGC0000777
Scanning BGC0000778
Scanning BGC0000779
Scanning BGC0000780
Scanning BGC0000781
Scanning BGC0000782
Scanning BGC0000783
Scanning BGC0000784
Scanning BGC0000785
Scanning BGC0000786
Scanning BGC0000787
Scanning BGC0000788
Scanning BGC0000789
Scanning BGC0000790
Scanning BGC0000791
Scanning BGC0000792
Scanning BGC0000793
Scanning BGC0000794
Scanning BGC0000795
Scanning BGC0000796
Scanning BGC0000797
Scanning BGC0000798
Scanning BGC0000799
Scanning BGC0000800
Scanning BGC0000801
Scanning BGC0000802
Scanning BGC0000803
Scanning BGC0000804
Scanning BGC0000805
Scanning BGC0000806
Scanning BGC0000807
Scanning BGC0000808
Scanning BGC0000809
Scanning BGC0000810
Scanning BGC0000811
Scanning BGC0000812
Scanning BGC0000813
Scanning BGC0000814
Scanning BGC0000815
Scanning BGC0000816
Scanning BGC0000817


Scanning BGC0001166
Scanning BGC0001167
[ripp] Wrong sequence information for afk79987.1
[ripp] Wrong sequence information for afk79996.1
[ripp] Wrong sequence information for afk79988.1
Scanning BGC0001168
Scanning BGC0001169
Scanning BGC0001170
[ripp] Can't find CDS feature for clospo_01168
Scanning BGC0001171
Scanning BGC0001172
Scanning BGC0001173
[ripp] Can't find CDS feature for bat_4209
Scanning BGC0001174
Scanning BGC0001176
[ripp] Wrong sequence information for lp52_05076
Scanning BGC0001177
Scanning BGC0001178
Scanning BGC0001179
Scanning BGC0001180
Scanning BGC0001181
Scanning BGC0001182
Scanning BGC0001183
Scanning BGC0001184
Scanning BGC0001185
Scanning BGC0001186
Scanning BGC0001187
Scanning BGC0001188
Scanning BGC0001189
Scanning BGC0001190
Scanning BGC0001191
Scanning BGC0001192
Scanning BGC0001193
Scanning BGC0001194
Scanning BGC0001195
Scanning BGC0001196
Scanning BGC0001197
Scanning BGC0001198
Scanning BGC0001199
Scanning BGC0001200
Scanning BGC0001201
Scanning BGC00

Scanning BGC0001524
Scanning BGC0001525
Scanning BGC0001526
Scanning BGC0001527
Scanning BGC0001528
Scanning BGC0001529
Scanning BGC0001530
Scanning BGC0001531
Scanning BGC0001532
Scanning BGC0001533
Scanning BGC0001534
Scanning BGC0001535
Scanning BGC0001536
Scanning BGC0001537
Scanning BGC0001538
Scanning BGC0001539
Scanning BGC0001540
Scanning BGC0001541
Scanning BGC0001542
Scanning BGC0001543
Scanning BGC0001544
Scanning BGC0001545
Scanning BGC0001546
Scanning BGC0001547
Scanning BGC0001548
Scanning BGC0001549
Scanning BGC0001550
Scanning BGC0001551
Scanning BGC0001552
Scanning BGC0001553
Scanning BGC0001554
Scanning BGC0001555
Scanning BGC0001556
Scanning BGC0001557
Scanning BGC0001558
Scanning BGC0001559
Scanning BGC0001560
Scanning BGC0001561
Scanning BGC0001562
Scanning BGC0001563
Scanning BGC0001564
Scanning BGC0001565
Scanning BGC0001566
Scanning BGC0001567
Scanning BGC0001568
Scanning BGC0001569
Scanning BGC0001570
Scanning BGC0001571
Scanning BGC0001572
Scanning BGC0001573


In [9]:
retired_bgcs = {}

with open("../../preprocessed/reports/p7-removed_extra_genes.tsv", "w") as o:
    o.write("bgc_id\tgene_ids\n")
    for bgc_id in removed_extra_genes:
        o.write("{}\t{}\n".format(bgc_id, ",".join(removed_extra_genes[bgc_id])))
        # uncommented: we don't need to retire BGCs because we trimmed out extra_genes
        #if bgc_id not in retired_bgcs:
        #    retired_bgcs[bgc_id] = set()
        #retired_bgcs[bgc_id].add("extra_genes_removed")
    
with open("../../preprocessed/reports/p7-removed_annotations.tsv", "w") as o:
    o.write("bgc_id\tgene_ids\n")
    for bgc_id in removed_annotations:
        o.write("{}\t{}\n".format(bgc_id, ";".join(removed_annotations[bgc_id])))
        if bgc_id not in retired_bgcs:
            retired_bgcs[bgc_id] = set()
        retired_bgcs[bgc_id].add("annotations_removed")
    
with open("../../preprocessed/reports/p7-removed_operons.tsv", "w") as o:
    o.write("bgc_id\tindexes\n")
    for bgc_id in removed_operons:
        o.write("{}\t{}\n".format(bgc_id, ",".join(removed_operons[bgc_id])))
        if bgc_id not in retired_bgcs:
            retired_bgcs[bgc_id] = set()
        retired_bgcs[bgc_id].add("operons_removed")

with open("../../preprocessed/reports/p7-no_gbk.tsv", "w") as o:
    o.write("bgc_id\taccession\n")
    for bgc_id in no_gbk:
        o.write("{}\t{}\n".format(bgc_id, no_gbk[bgc_id]))
        if bgc_id not in retired_bgcs:
            retired_bgcs[bgc_id] = set()
        retired_bgcs[bgc_id].add("no_gbk")
    
with open("../../preprocessed/reports/p7-no_taxonomy.tsv", "w") as o:
    o.write("bgc_id\torganism\n")
    for bgc_id in no_taxid:
        o.write("{}\t{}\n".format(bgc_id, no_taxid[bgc_id]))
        if bgc_id not in retired_bgcs:
            retired_bgcs[bgc_id] = set()
        retired_bgcs[bgc_id].add("no_taxonomy")
        
        
with open("../../preprocessed/reports/p7-retired_list.tsv", "w") as o:
    for bgc_id in retired_bgcs:
        o.write("{}\t{}\n".format(bgc_id, ";".join(retired_bgcs[bgc_id])))
    
with open("../../preprocessed/reports/p7-added_mibig_genes.tsv", "w") as o:
    o.write("bgc_id\tgenes\n")
    for bgc_id in added_mibig_genes:
        o.write("{}\t{}\n".format(bgc_id, ",".join(added_mibig_genes[bgc_id])))

with open("../../preprocessed/reports/p7-ripps_precursors_to_fix.tsv", "w") as o:
    for bgc_id in ripps_to_fix:
        o.write("{}\t{}\n".format(bgc_id, ",".join([str(idx) for idx in ripps_to_fix[bgc_id]])))

In [10]:
### update fixed list ###
with open("../../preprocessed/reports/p7-fixed_list.tsv", "w") as o:
    for bgc_id in fixed:
        o.write("{}\t{}\n".format(bgc_id, ";".join(fixed[bgc_id])))